In [ ]:
import torch
import torchvision
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

print(torch.__version__)

# Preprocessing and loading the dataset
temp_transform = transforms.ToTensor()
temp_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=temp_transform)
temp_loader = DataLoader(temp_trainset, batch_size=128, shuffle=False, num_workers=2)

mean = torch.zeros(3)
std = torch.zeros(3)
no_of_batches = 0

for x, y in temp_loader:
    mean += x.mean(dim=[0,2,3])
    std += x.std(dim=[0,2,3])
    no_of_batches += 1

mean /= no_of_batches
std /= no_of_batches
print("Mean: ", mean)
print("Std: ", std)


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean.tolist(), std.tolist())
])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


# Num_workers depends on system cores
train_loader = DataLoader(cifar_trainset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar_testset, batch_size=128, shuffle=False, num_workers=2)

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Starting dimensions 32 x 32 x 64 -> 16 x 16 x 128
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv1_3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bnorm1_1 = nn.BatchNorm2d(64)
        self.bnorm1_2 = nn.BatchNorm2d(64)
        self.bnorm1_3 = nn.BatchNorm2d(64)
        
        # Starting dimensions 16 x 16 x 128 -> 8 x 8 x 128
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3 ,stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv2_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bnorm2_1 = nn.BatchNorm2d(128)
        self.bnorm2_2 = nn.BatchNorm2d(128)
        self.bnorm2_3 = nn.BatchNorm2d(128)


        # Starting dimensions 8 x 8 x 256 -> 4 x 4 x 256
        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bnorm3_1 = nn.BatchNorm2d(256)
        self.bnorm3_2 = nn.BatchNorm2d(256)
        self.bnorm3_3 = nn.BatchNorm2d(256)

        # Reduces dimension by half (32 x 32 -> 16 x 16)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Activation function
        self.relu = nn.ReLU()

        # Final fully connected layer, input (4 x 4 x 256) dimensions should be flattened
        self.fully_connected_layer = nn.Linear(4*4*256, 10)
        # Cross entropy loss
        self.cross_entropy_loss = nn.CrossEntropyLoss()


    def forward(self, input):
        input = self.relu(self.bnorm1_1(self.conv1_1(input)))
        input = self.relu(self.bnorm1_2(self.conv1_2(input)))
        input = self.relu(self.bnorm1_3(self.conv1_3(input)))
        input = self.maxpool(input)

        input = self.relu(self.bnorm2_1(self.conv2_1(input)))
        input = self.relu(self.bnorm2_2(self.conv2_2(input)))
        input = self.relu(self.bnorm2_3(self.conv2_3(input)))
        input = self.maxpool(input)

        input = self.relu(self.bnorm3_1(self.conv3_1(input)))
        input = self.relu(self.bnorm3_2(self.conv3_2(input)))
        input = self.relu(self.bnorm3_3(self.conv3_3(input)))
        input = self.maxpool(input)

        input = self.fully_connected_layer(torch.flatten(input, 1, -1))

        return input

In [ ]:
import torch.optim as optim
cnn_model = CNN()
optimiser = optim.Adam(cnn_model.parameters(), lr=0.001)
loss_history = []
epochs = 100
total_loss = 0
no_of_batches = 0


for i in range(epochs):
    total_loss = 0
    no_of_batches = 0
    for x_train, y_train in train_loader:
        optimiser.zero_grad()
        output = cnn_model(x_train)
        loss = cnn_model.cross_entropy_loss(output, y_train)
        loss.backward()
        optimiser.step()
        no_of_batches += 1
        total_loss += loss.item()
    avg_loss = total_loss / no_of_batches
    loss_history.append(avg_loss)
    print("Epoch" + str(i) + "average_loss : " + str(avg_loss))

In [ ]:
total_correct = 0
total_samples = 0
for x_test, y_test in test_loader:
    y_pred = torch.argmax(cnn_model(x_test), dim=1)
    correct_pred = torch.sum(y_pred == y_test)
    batch_accuracy = correct_pred / len(x_test)
    total_correct += correct_pred
    total_samples += len(x_test)
accuracy = total_correct / total_samples